Descobrindo a senha com DEAP
============================



## Importações



In [1]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

from funcoes import gene_letra

## Problema da senha usando `DEAP`



Primeiro definimos as constantes.



In [2]:
# relacionadas ao problema a ser resolvido
SENHA = "correcthorsebatterystaple"
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz"
NUM_GENES = len(SENHA)

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.15
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Precisamos definir a função objetivo também.



In [3]:
def funcao_objetivo_senha(individuo, senha_verdadeira):
    """Computa a funcao objetivo de um individuo no problema da senha

    Args:
      individiuo: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir

    Returns:
      Uma tupla com a  "distância" entre a senha proposta e a senha verdadeira.
      Essa distância é medida letra por letra. Quanto mais distante uma letra
      for da que deveria ser, maior é essa distância.
    """
    diferenca = 0

    for letra_candidato, letra_oficial in zip(individuo, senha_verdadeira):
        diferenca = diferenca + abs(ord(letra_candidato) - ord(letra_oficial))

    return (diferenca, )

O `DEAP` não tem uma função de mutação de letras, precisamos usar a que nós criamos na disciplina. Tem um pequeno porém, o `DEAP` espera que a função de mutação retorne um indivíduo dentro de uma tupla. Não sei dizer o motivo dessa escolha, temos que aceitar e seguir.



In [4]:
def mutacao_senha(individuo, letras):
    """Realiza a mutação de um gene no problema da senha.

    Args:
      individuo: uma lista representado um individuo no problema da senha
      letras: letras possíveis de serem sorteadas.

    Return:
      Uma tupla de um individuo (senha) com um gene mutado.
    """
    gene = random.randint(0, len(individuo) - 1)
    individuo[gene] = gene_letra(letras)
    return (individuo, )

O restante não tem segredo. Único detalhe é que agora o problema é de minimização, então precisamos alterar isso.



In [5]:
# Devemos alterar aqui por ser um problema de minimização
creator.create("Fitness_min", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness_min)

toolbox = base.Toolbox()

toolbox.register("cria_gene", gene_letra, LETRAS_POSSIVEIS)
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.cria_gene,
    NUM_GENES,
)
toolbox.register(
    "population", tools.initRepeat, list, toolbox.individual, TAMANHO_POP
)
toolbox.register("evaluate", funcao_objetivo_senha, SENHA)
toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register(
    "mutate",
    mutacao_senha,
    letras=LETRAS_POSSIVEIS
)

hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.population()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_CRUZAMENTO,
    mutpb=CHANCE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

print()
print("Hall da fama:")
print("".join(hall_da_fama.items[0]))

gen	nevals	avg   	std    	min	max
0  	50    	218.62	31.2915	161	279
1  	31    	189.78	24.9602	145	255
2  	30    	171.86	19.8837	129	221
3  	27    	159.18	14.8306	122	196
4  	30    	149.28	11.3279	122	178
5  	38    	140.78	13.6151	106	174
6  	30    	128.46	9.68341	105	147
7  	29    	119.14	10.7331	100	143
8  	30    	111.98	9.37548	95 	137
9  	32    	106.74	7.23826	94 	129
10 	27    	101.98	7.70582	81 	126
11 	28    	96.38 	7.36177	81 	112
12 	27    	91.54 	7.13361	81 	115
13 	22    	86.1  	5.92368	81 	112
14 	26    	83.12 	3.88144	75 	100
15 	28    	82.48 	6.09669	77 	104
16 	35    	80.06 	4.21146	71 	92 
17 	27    	77.46 	3.69437	71 	90 
18 	31    	76.02 	5.29713	68 	100
19 	35    	73.24 	3.91183	64 	84 
20 	33    	70.5  	4.11947	62 	79 
21 	26    	69.32 	6.10718	58 	87 
22 	36    	64.58 	3.91964	58 	79 
23 	28    	62.7  	4.90408	52 	80 
24 	37    	60.48 	5.97073	46 	84 
25 	31    	56.36 	3.65382	49 	69 
26 	28    	54.04 	4.01477	48 	73 
27 	26    	52.34 	3.68027	47 	66 
28 	23    	51.